<a href="https://colab.research.google.com/github/Sanjaya-Shastry/tarantino-LLM--feed-forward-NN-/blob/main/LLM_0(tarantino).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!wget https://raw.githubusercontent.com/Sanjaya-Shastry/tarantino-LLM--feed-forward-NN-/refs/heads/main/scripts.txt

--2025-07-13 12:21:17--  https://raw.githubusercontent.com/Sanjaya-Shastry/tarantino-LLM--feed-forward-NN-/refs/heads/main/scripts.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 329361 (322K) [text/plain]
Saving to: ‘scripts.txt.6’

scripts.txt.6       100%[===================>] 321.64K  --.-KB/s    in 0.04s   

2025-07-13 12:21:17 (7.14 MB/s) - ‘scripts.txt.6’ saved [329361/329361]



In [34]:
import numpy as np
from tensorflow import keras
from keras import layers

In [35]:
with open('scripts.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [36]:
text = text.lower()

In [37]:
lines = text.splitlines()
cleaned_lines = [line.strip() for line in lines if line.strip() != '']
cleaned_text = ' '.join(cleaned_lines)

In [38]:
cleaned_text[:100]

"django unchained written by quentin tarantino i ext - countryside - broiling hot day as the film's o"

In [39]:
import string

allowed_chars = set(string.ascii_letters + string.digits + " .,")
cleaned_text= ''.join(char for char in cleaned_text if char in allowed_chars)

In [40]:
cleaned_text[:100]

'django unchained written by quentin tarantino i ext  countryside  broiling hot day as the films open'

In [41]:
#tokenization

chars = sorted(set(cleaned_text))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for ch, i in char_to_int.items()}
vocab_size = len(chars)

In [42]:
encoded_text = np.array([char_to_int[i] for i in cleaned_text], dtype=np.int32)

In [43]:
len(encoded_text)

361467

In [44]:
context_size=16
X=[]
y=[]

for i in range(len(encoded_text) - context_size):
    X.append(encoded_text[i:i+context_size])
    y.append(encoded_text[i+context_size])


In [45]:
X= np.array(X)
y= np.array(y)

In [48]:
#We want to convert these into vectors of dimension of len(vocab_size) i.e 39
#embeddings

X_embedded = np.zeros((len(X), context_size, vocab_size), dtype=np.float32)
y_embedded = np.zeros((len(X), vocab_size), dtype=np.float32)

for i in range(len(X)):
    for t in range(context_size):
        word_index = X[i][t]
        X_embedded[i, t, word_index] = 1
    y_embedded[i, y[i]] = 1


In [49]:
X_flat = X_embedded.reshape(len(X), context_size * vocab_size)


In [50]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Input(shape=(context_size * vocab_size,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(vocab_size, activation='softmax')
    ])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
model.fit(X_flat, y_embedded, batch_size=128, epochs=10, validation_split=0.25)


Epoch 1/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 34s 16ms/step - accuracy: 0.3615 - loss: 2.2175 - val_accuracy: 0.4310 - val_loss: 1.8941
Epoch 2/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 35s 16ms/step - accuracy: 0.5182 - loss: 1.5902 - val_accuracy: 0.4601 - val_loss: 1.8216
Epoch 3/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.5723 - loss: 1.3942 - val_accuracy: 0.4653 - val_loss: 1.8144
Epoch 4/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.6071 - loss: 1.2709 - val_accuracy: 0.4640 - val_loss: 1.8396
Epoch 5/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 35s 16ms/step - accuracy: 0.6380 - loss: 1.1681 - val_accuracy: 0.4681 - val_loss: 1.8791
Epoch 6/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 32s 15ms/step - accuracy: 0.6642 - loss: 1.0745 - val_accuracy: 0.4644 - val_loss: 1.9707
Epoch 7/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 35s 16ms/step - accuracy: 0.6845 - loss: 1.0053 - val_accuracy: 0.4640 - val_loss: 2.0386
Epoch 8/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.7054 -

In [52]:
model.save("LLM_0(tarantino).keras")

In [53]:
def generate_text(seed, length=300):
    result = seed

    for i in range(length):
        cleaned_seed = ''.join([c for c in result.lower() if c in char_to_int])
        seed_len = len(cleaned_seed)

        if seed_len > context_size:
            context = cleaned_seed[-context_size:]
        elif seed_len < context_size:
            padding = ' ' * (context_size - seed_len)
            context = padding + cleaned_seed
        else:
            context = cleaned_seed

        input_seq = [char_to_int.get(c, 0) for c in context]

        x = np.zeros((1, context_size, vocab_size), dtype=np.float32)
        for j, char_index in enumerate(input_seq):
            x[0, j, char_index] = 1

        x_flat = x.reshape(1, context_size * vocab_size)
        preds = model.predict(x_flat, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = int_to_char[next_index]
        result += next_char

    return result


In [56]:
generate_text('Django was a',length=50)

'Django was a little bit be fuck i dooth, and smilly love count'